In [62]:
import math
import numpy as np
import csv
import matplotlib.pyplot as plt
import time
with open('q3_test_dataset.csv','r') as csvfile:
    dataset = list(csv.reader(csvfile))
my_list = []
for i in dataset:
    my_list.append(i)

with open('q3_train_dataset.csv','r') as csvfile:
    dataset_train = list(csv.reader(csvfile))
my_list_train = []
for i in dataset_train:
    my_list_train.append(i)

#Adding dummy variable
data_test = np.array(my_list)
data_train = np.array(my_list_train)

rec = []
data_test = data_test[1:,:]
data_train = data_train[1:,:]
full = np.append(data_test,data_train,axis = 0)
fulldata = np.zeros((data_train.shape[0] + data_test.shape[0],15)) #creating full data first 179 test last 712 train
fulldata = fulldata.astype('str')
c = 0
for row in full:

    if row[1] == '1':
        rec = [1.0,0.0,0.0]
    elif row[1] == '2':
        rec = [0.0,1.0,0.0]
    elif row[1] == '3':
        rec = [0.0,0.0,1.0]

    if row[2] == "male":
        rec.append(0.0)
    else:
        rec.append(1.0)

    if row[7] == "C":
        rec.append(1.0)
        rec.append(0.0)
        rec.append(0.0)
    elif row[7] == "Q":
        rec.append(0.0)
        rec.append(1.0)
        rec.append(0.0)
    elif row[7] == "S":
        rec.append(0.0)
        rec.append(0.0)
        rec.append(1.0)
    rec = np.array(rec)
    fulldata[c] = np.append(full[c],rec)
    rec = []
    c = c + 1
print(fulldata[0])
fulldata = np.delete(fulldata,[1,2,7],axis = 1)
fulldata = fulldata.astype('float64')
print(fulldata[0])
#normalizing colmns
col_index = 0
for col in fulldata.T:
    fulldata[:,col_index] = (fulldata[:,col_index] - np.min(col)) / (np.max(col) - np.min(col))
    col_index = col_index + 1
x_test = fulldata[:179,1:]
x_train = fulldata[179:,1:]
y_test = fulldata[:179,0].reshape(179,1)
y_train = fulldata[179:,0].reshape(712,1)

['0' '3' 'male' '4' '3' '2' '27.9' 'S' '0.0' '0.0' '1.0' '0.0' '0.0' '0.0'
 '1.0']
[ 0.   4.   3.   2.  27.9  0.   0.   1.   0.   0.   0.   1. ]


In [53]:
beta = np.random.normal(0,0.01,11).reshape(11,1)
alpha = np.array([0.0001,0.001,0.1])
    
#full batch gradient descent
def full_batch(x_train,y_train,x_test,alpha,max_iter):
    beta = np.random.normal(0,0.01,11).reshape(11,1)
    b0 = np.random.normal(0,0.01,1)
    for i in range(max_iter):
        z = x_train @ beta
        z = z + b0
        sigmz = 1 / (1 + np.exp(-z))
        grad = ((y_train - sigmz).T @ x_train).reshape(11,1) 
        grad0 = np.sum(y_train - sigmz)

        beta = beta + alpha*grad
        b0 = b0 + alpha*grad0
        if((i+1) % 100 == 0):
            print(f'Model Weights in iteration {i+1} = {np.insert(beta,0, b0)}')
            print()
    
    y_batch = b0 + x_test @ beta
    y_batch = (y_batch >= 0).astype(int)
    return y_batch

# acc_batch = (y_batch == y_test).all(axis=1).mean()
# print(acc_batch)


In [54]:
#stochastic gradient descent
def stoch(x_train,y_train,x_test,alpha,max_iter):
    beta_s = np.random.normal(0,0.01,11).reshape(11,1)
    b0_s = np.random.normal(0,0.01,1)
    for i in range(max_iter):
        rand_ind = np.random.randint(712)
        x_rand = x_train[rand_ind]
        y_rand = y_train[rand_ind]
        z = x_rand @ beta_s
        z = z + b0_s
        sigmz = 1 / (1 + np.exp(-z))
        grad = ((y_rand - sigmz) * x_rand).reshape(11,1)
        grad0 = y_rand - sigmz
        beta_s = beta_s + alpha * grad
        b0_s = b0_s + alpha * grad0
    
    y_s = b0_s + x_test @ beta_s
    y_s = (y_s >= 0).astype(int)
    return y_s

# acc_s = (y_s == y_test).all(axis=1).mean()
# print(acc_s)

In [55]:
#mini batch gradient descent
def mini_batch(x_train,y_train,x_test,alpha,max_iter):
    beta_32 = np.random.normal(0,0.01,11).reshape(11,1)
    beta_32_0 = np.random.normal(0,0.01,1)
    for i in range(max_iter):
    
        x_train_32 = x_train[i:i+32,:]
        y_train_32 = y_train[i:i+32,:]
        z = x_train_32 @ beta_32
        z = z + beta_32_0
        sigmz = 1 / (1 + np.exp(-z))
        grad = ((y_train_32 - sigmz).T @ x_train_32).reshape(11,1)
        grad0 = np.sum(y_train_32 - sigmz)
        beta_32 = beta_32 + alpha*grad
        beta_32_0 = beta_32_0 + alpha * grad0
    
    y_mini_batch = beta_32_0 + x_test @ beta_32
    y_mini_batch = (y_mini_batch >= 0).astype(int) 
    return y_mini_batch

In [43]:
def get_conf(y_predict, y_test):
    
    TP = 0
    FN = 0
    FP = 0
    TN = 0
    
    for i in range(y_predict.shape[0]):
        if y_predict[i] == y_test[i]:
            if(y_predict[i] == 1):
                TP += 1
            else:
                TN += 1
        else:
            if(y_predict[i] == 0):
                FN += 1
            else:
                FP += 1
                
    conf_matrix = np.zeros((2,2))
    conf_matrix[0,0] = TP
    conf_matrix[0,1] = FP
    conf_matrix[1,0] = FN
    conf_matrix[1,1] = TN
    
    if(TP+FP+FN+TN == 0):
        acc = 0
    else:
        acc = (TP+TN)/(TP+FP+FN+TN)
        
    if(FP+TN == 0):
        FPR = 0
    else:    
        FPR = FP/(FP+TN)
         
    if(TP+FP == 0):
        prec = 0
    else:        
        prec =  TP/(TP+FP)
             
    if(TP+FN == 0):
        recall = 0
    else:    
        recall = TP/(TP+FN)
              
    if(TN+FN == 0):
        NPV = 0
    else:   
        NPV = TN/(TN+FN)
            
    if(FP+TP == 0):
        FDR = 0
    else:     
        FDR = FP/(FP+TP)
             
    if(prec+recall == 0):
        F1 = 0
    else:    
        F1 = (2*recall*prec)/(prec+recall)
             
    if(prec+recall == 0):
        F2 = 0
    else:    
        F2 = (5*prec*recall)/(4*prec+recall)
        
    return conf_matrix,acc,FPR,prec,recall,NPV,FDR,F1,F2


In [47]:
learningRate = np.array([0.0001,0.001,0.01])
times = []
#Mini batch calculations
for i in range(3):
    start = time.time()
    y_mini_batch = mini_batch(x_train,y_train,x_test,learningRate[i],1000)
    stop = time.time()
    times.append(stop - start)
    conf_matrix,acc,FPR,prec,recall,NPV,FDR,F1,F2 = get_conf(y_mini_batch,y_test)
    print(f' Confusion Matrix of {learningRate[i]}, learning rate Mini Batch:\n{conf_matrix}')
    print(f' Accuracy:{acc}, FPR:{FPR},Precision:{prec},Recall:{recall},NPV:{NPV},FDR:{FDR},F1:{F1},F2:{F2}')
    print()
print(f' Iteration time Mini Batch 0.0001: {times[0]}, Iteration time Mini Batch 0.001: {times[1]},Iteration time Mini Batch 0.01: {times[2]}')

 Confusion Matrix of 0.0001, learning rate Mini Batch:
[[ 12.   0.]
 [ 57. 110.]]
 Accuracy:0.6815642458100558, FPR:0.0,Precision:1.0,Recall:0.17391304347826086,NPV:0.6586826347305389,FDR:0.0,F1:0.29629629629629634,F2:0.20833333333333334

 Confusion Matrix of 0.001, learning rate Mini Batch:
[[ 35.   1.]
 [ 34. 109.]]
 Accuracy:0.8044692737430168, FPR:0.00909090909090909,Precision:0.9722222222222222,Recall:0.5072463768115942,NPV:0.7622377622377622,FDR:0.027777777777777776,F1:0.6666666666666666,F2:0.5608974358974359

 Confusion Matrix of 0.01, learning rate Mini Batch:
[[ 45.   6.]
 [ 24. 104.]]
 Accuracy:0.8324022346368715, FPR:0.05454545454545454,Precision:0.8823529411764706,Recall:0.6521739130434783,NPV:0.8125,FDR:0.11764705882352941,F1:0.75,F2:0.6880733944954128

 Iteration time Mini Batch 0.0001: 0.042873382568359375, Iteration time Mini Batch 0.001: 0.021947622299194336,Iteration time Mini Batch 0.01: 0.017961740493774414


In [48]:
learningRate = np.array([0.0001,0.001,0.01])
times = []
#Stochastic Gradient Ascent Calculations
for i in range(3):
    start = time.time()
    y_stoch = stoch(x_train,y_train,x_test,learningRate[i],1000)
    stop = time.time()
    times.append(stop - start)
    conf_matrix,acc,FPR,prec,recall,NPV,FDR,F1,F2 = get_conf(y_stoch,y_test)
    print(f' Confusion Matrix of {learningRate[i]}, learning rate Stochastic GD:\n{conf_matrix}')
    print(f' Accuracy:{acc}, FPR:{FPR},Precision:{prec},Recall:{recall},NPV:{NPV},FDR:{FDR},F1:{F1},F2:{F2}')
    print()
print(f' Iteration time Stochastic 0.0001: {times[0]}, Iteration time Stochastic 0.001: {times[1]},Iteration time Stochastic 0.01: {times[2]}')

 Confusion Matrix of 0.0001, learning rate Stochastic GD:
[[ 25.   6.]
 [ 44. 104.]]
 Accuracy:0.7206703910614525, FPR:0.05454545454545454,Precision:0.8064516129032258,Recall:0.36231884057971014,NPV:0.7027027027027027,FDR:0.1935483870967742,F1:0.5,F2:0.4071661237785017

 Confusion Matrix of 0.001, learning rate Stochastic GD:
[[  0.   0.]
 [ 69. 110.]]
 Accuracy:0.6145251396648045, FPR:0.0,Precision:0,Recall:0.0,NPV:0.6145251396648045,FDR:0,F1:0,F2:0

 Confusion Matrix of 0.01, learning rate Stochastic GD:
[[ 31.   4.]
 [ 38. 106.]]
 Accuracy:0.7653631284916201, FPR:0.03636363636363636,Precision:0.8857142857142857,Recall:0.4492753623188406,NPV:0.7361111111111112,FDR:0.11428571428571428,F1:0.5961538461538461,F2:0.4983922829581994

 Iteration time Stochastic 0.0001: 0.01695418357849121, Iteration time Stochastic 0.001: 0.014991044998168945,Iteration time Stochastic 0.01: 0.014927387237548828


In [49]:
learningRate = np.array([0.0001,0.001,0.01])
times = []
#Stochastic Gradient Ascent Calculations
for i in range(3):
    start = time.time()
    y_full = full_batch(x_train,y_train,x_test,learningRate[i],1000)
    stop = time.time()
    times.append(stop - start)
    conf_matrix,acc,FPR,prec,recall,NPV,FDR,F1,F2 = get_conf(y_full,y_test)
    print(f' Confusion Matrix of {learningRate[i]}, learning rate Full Batch GA:\n{conf_matrix}')
    print(f' Accuracy:{acc}, FPR:{FPR},Precision:{prec},Recall:{recall},NPV:{NPV},FDR:{FDR},F1:{F1},F2:{F2}')
    print()
print(f' Iteration time Full Batch GA 0.0001: {times[0]}, Iteration time Full Batch GA 0.001: {times[1]},Iteration time Full Batch GA 0.01: {times[2]}')

Model Weights in iteration 100 = [-0.24375706 -0.13923328 -0.02254757  0.03143011  0.05950121  0.24034707
  0.04790939 -0.50180202  0.62504662  0.0791227  -0.01989099 -0.28803939]

Model Weights in iteration 200 = [-0.29247748 -0.23072401 -0.03358422  0.0438601   0.09893254  0.38447158
  0.07019412 -0.71693167  1.08705887  0.10838215 -0.02023675 -0.3656735 ]

Model Weights in iteration 300 = [-0.33052324 -0.31874418 -0.04719663  0.04766288  0.12572671  0.46726396
  0.0744861  -0.84206179  1.41264633  0.10993924 -0.01723236 -0.40828074]

Model Weights in iteration 400 = [-0.35953758 -0.40171611 -0.06261278  0.04581045  0.14626847  0.52207716
  0.07121033 -0.92261357  1.65049387  0.10181934 -0.0137413  -0.43266624]

Model Weights in iteration 500 = [-0.38046706 -0.47946743 -0.0792648   0.04013403  0.16339998  0.56292263
  0.06513224 -0.97831043  1.82854523  0.09105432 -0.01044824 -0.44612377]

Model Weights in iteration 600 = [-0.39470503 -0.55253502 -0.09677717  0.03179046  0.17847295  

In [56]:
y_stoch,beta,beta0 = full_batch(x_train,y_train,x_test,0.01,1000)

Model Weights in iteration 100 = [ 0.60510926 -3.45934947 -1.4621866  -0.9237445   0.95014831  1.4250249
  0.29025017 -1.12138751  3.3420412   0.21610901  0.3452407   0.0386037 ]

Model Weights in iteration 200 = [ 0.74494531 -4.14386449 -2.09025607 -1.2379585   1.39240476  1.48759645
  0.32855571 -1.08242856  3.29813368  0.238606    0.40852182  0.09266164]

Model Weights in iteration 300 = [ 0.76709078 -4.29122108 -2.34324511 -1.30988706  1.68326416  1.48274737
  0.33862601 -1.0655043   3.28542976  0.23823613  0.41919566  0.10450315]

Model Weights in iteration 400 = [ 0.76705691 -4.32268192 -2.45370807 -1.32848693  1.87495253  1.47013154
  0.3422961  -1.05659243  3.280756    0.23471828  0.42008123  0.10710156]

Model Weights in iteration 500 = [ 0.76362276 -4.32819707 -2.50589668 -1.33699315  2.0016066   1.45965132
  0.3440646  -1.05131487  3.27854142  0.23169673  0.41924101  0.10752918]

Model Weights in iteration 600 = [ 0.76046768 -4.32802368 -2.53230267 -1.34337507  2.0857192   1

In [63]:
print(beta)



[[-4.32432069]
 [-2.56209256]
 [-1.35792714]
 [ 2.22189093]
 [ 1.43953821]
 [ 0.34660938]
 [-1.04282729]
 [ 3.27558101]
 [ 0.22604504]
 [ 0.41635922]
 [ 0.10698192]]


In [60]:
x_test[0]

array([0.04498618, 0.375     , 0.33333333, 0.05445717, 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       1.        ])